# Practical 1
https://fleuret.org/ee559/ee559-practical-1.pdf

In [1]:
import torch
import time
import numpy as np

## 1 Creating a garbage tensor
We test different approach to slicing operators:
1. Addition : add 1d tensor to a slice (line or column)
    - The 1d tensor is automatically broadcasted to the convenient configuration (eg. does not matter if line or column) (#3,#5)
2. Assigning : replace the value
   - 1D tensor ( #2, #4) 
   -  Select (dim,index) (#5)
   - narrow (dim,start,lenght) (#9,#10)
3. Incrementation (+=)with a constant (#1)
    - Does not work with a 1d tensor (no broadcast?)
4. Multiplication
    - Works with a tensor #8
    - a constant #9, #10
5. Using the subtensor (narrow)
    - Difference between sliced tensor (d[...]) and narrow (new tensor). We can't change the value in the new tensor and expect a change in the new one !
    - Broadcasted to good dimensions ( #10)



In [158]:
d=torch.ones(13,13)
l2=torch.full((1,13),2)
l1=torch.full((1,13),1)
c3=torch.full((2,2),3)
d[1]+=1                  #1
d[:,1]=l2                #2
d[6]=d[6]+l1             #3
d[:,6]=d[6]              #4
d[11]=d[11]+l1           #5
d[:,11]=d.select(0,11)   #6
d[3:5,3:5]=c3            #7
d[8:10,3:5]=d[8:10,3:5]*c3#8
d[8:10,8:10]=3*d.narrow(0,8,2).narrow(1,8,2) #9
d[3:5,8:10]=l1.narrow(1,0,2)*3 #10



In [161]:
d

tensor([[1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 3., 2.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [2., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 2.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [2., 3., 2., 2., 2., 2., 3., 2., 2., 2., 2., 2., 2.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.]])

## 2 Eigendecomposition

In [2]:
mean=torch.zeros((20,20))
std=torch.ones((20,20))
d=torch.normal(mean,std)
eig=torch.eig(d)
eig

(tensor([[ 5.3293,  0.0000],
         [ 4.6553,  1.8756],
         [ 4.6553, -1.8756],
         [-0.7595,  3.9994],
         [-0.7595, -3.9994],
         [ 0.1258,  3.6287],
         [ 0.1258, -3.6287],
         [-3.9391,  1.5210],
         [-3.9391, -1.5210],
         [ 2.7112,  0.0000],
         [ 1.7258,  1.8512],
         [ 1.7258, -1.8512],
         [-2.9447,  1.3800],
         [-2.9447, -1.3800],
         [-2.2245,  1.0430],
         [-2.2245, -1.0430],
         [ 0.7441,  1.1303],
         [ 0.7441, -1.1303],
         [-1.5948,  0.0000],
         [ 0.2306,  0.0000]]), tensor([]))

## 3 Flops per second
Best complexity for NxN matricial product = n^2.37 (coppersmith-winograd algo)

In [2]:
size=5000
mean=torch.zeros((size,size))
std=torch.ones((size,size))
m1=torch.normal(mean,std)
m2=torch.normal(mean,std)
t1=time.perf_counter()
prod=torch.mm(m1,m2)
tf=time.perf_counter()-t1
print(tf, "seconds elapsed")

0.7844509230008043 seconds elapsed


In [60]:
complexity=pow(size,2.37)
flop=complexity/tf
print (tf,"seconds",int(complexity),"operations,",int(flop/1000000),"Mflops")

0.7844509230008043 seconds 584196445 operations, 744 Mflops


## 4 Playing with strides

We evaluate 3 different methods for multiplexing a tensor:
1. Multiply each row by its index+1 (with a for loop)
2. Add the previous row with the first row (with for loop)
3. Element-wise multiplication with a tensor formed by the row indices

In [54]:
def mul_row(tensor):
    row,col=tensor.size()
    for id_row in range(row):
        tensor[id_row]=tensor[id_row]*(id_row+1)
    return tensor
def mul_row_add(tensor):
    row,col=tensor.size()
    for id_row in range(1,row):
        tensor[id_row]=tensor[id_row-1]+tensor[0]
    return tensor
def mul_row_fast(tensor):
    row,col=tensor.size()
    mul_coeff=torch.arange(1.,float(row+1)).view(row,1).expand(row,col)
    return tensor*mul_coeff
    
d=torch.full((1000,400),2)

In [56]:
t1=time.perf_counter()
res=mul_row(d)
t_line=time.perf_counter()-t1

In [51]:
t1=time.perf_counter()
res=mul_row_add(d)
t_add=time.perf_counter()-t1


In [57]:
t1=time.perf_counter()
res=mul_row_fast(d)
t_fast=time.perf_counter()-t1

In [59]:
print("Linewise multiplication in ",int(t_line*1e6)," useconds. \nLinewise addition in ",int(t_add*1e6)," useconds. \nFast multiplex in ",int(t_fast*1e6),"useconds")

Linewise multiplication in  17027  useconds. 
Linewise addition in  13505  useconds. 
Fast multiplex in  800 useconds
